In [1]:
#Overland Flow Assignment
#05/08/2021

import pandas as pd
import math
ObservedUS = pd.read_excel (r'C:\Users\AnnaA\Downloads\unsteady.xls')
print(ObservedUS)

    Unsteady Plot       Unnamed: 1  Unnamed: 2
0   Slope = 2.83%          Width =       200.0
1             NaN        Length =        185.0
2             NaN          Area =      37000.0
3      Time - min  Flow Rate (cfs)         NaN
4               0                0         NaN
5              10                0         NaN
6              20            0.132         NaN
7              30             0.67         NaN
8              40            0.782         NaN
9              50            0.388         NaN
10             60            0.165         NaN
11             70            0.078         NaN
12             80             0.04         NaN
13             90            0.022         NaN


In [27]:
import plotly

#timeforei=[0,142,288,499,781,1063,1372,1969,2874,3770,4679,5566,6463,6639,6815,6990,7166,7342]#seconds
#ieobserved=[0,0, 0.1715973003,0.9412239981,1.189849205,1.236509958,1.313791831,1.261240157,1.370913215,1.401377953,1.433227452,1.370913215,1.370913215,0.8773849319,0.4935285823,0.219346233,0.05483640722,0]#mm/hr
#fig = dict({
#    "data": [{"type": "scatter",
#              "x": timeforei,
#              "y": ieobserved}],
#    "layout": {"title": {"text": "Rainfall intensity"}}
#})

# To display the figure defined by this dict, use the low-level plotly.io.show function
import plotly.io as pio

#pio.show(fig)

def estimateie(timetocompare):
    if (0/60) <= timetocompare <(14/60): #diveded by 60 to get from minutes to hours
        i=(.8*25.4) #m/hr
        f=(.8*25.4) #m/hr
    elif (14/60) <= timetocompare < (28/60):#diveded by 60 to get from minutes to hours
        i=(3.3*25.4)
        f=(2**(-0.050*((timetocompare*60)-14))*25.4) #m/hr
    else:
    #elif (28/60) <= timetocompare < (40/60):#diveded by 60 to get from minutes to hours
        i=(.6*25.4)#m/hr
        f=(.6*25.4)#m/hr
    ie=i-f #m/hr 
    return ie
    
estimateie(.44)

67.29295843588744

In [28]:
#Input parameters:

#from assignment handout:
S = 0.0283 #38.4%
width = 60.96 #m (converted from 200 ft)
length = 56.388 #m (converted from 185.0 ft)
#rainfallintensity = 83.82 #mm/h (3.3 in/hr)
duration = 1.5 #hours

#mannings eqn:
β=3/5
Cn=1 #bc we're in metric
n = 2 #in part 1
α = (n/(Cn*S**.5))**β
m=5/3

In [29]:
import plotly.express as px
fig = px.line(ObservedUS, x="Unsteady Plot", y="Unnamed: 1", title="inflow hydrograph")
fig.show()

In [30]:
a=Cn*S**.5/n
#ie = 292.1 #mm/hr (from part 2 instructions)
#ie = 490.22 #mm/hr (from part 1 instructions)

In [31]:
#Continued Parameter info
#=============================================================================
#making a floatable range
#=============================================================================
def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step
#============================================================================
#time domain
Δt=0.00555556 #hrs (converted from 20 seconds)
time=[]
for i in frange(0,duration,Δt):
    time.append(i)

#============================================================================
#space domain
Δx=3.048 #m (converted from 10 ft)
space = []
for i in frange(0,length,Δx):
    space.append(i)
print(len(space))
a=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
len(a)

19


18

In [21]:
newflow = {0:[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}
timecounter = 0
referencetime =0
for j in range(0,len(time)-1):
    timecounter+=Δt
    newspaceflow=[0]
    for i in range(0,len(space)-1):
        qj1i=newspaceflow[i] #q^j+1,i
        qji1=newflow[(referencetime)][i] #q^j,i+1
        starter=(qj1i+qji1)/2
        if starter!=0:
            middleterm=starter**(β-1)
            qnext=(qj1i*(Δt/Δx)+α*β*qji1*middleterm+Δt*(estimateie(timecounter)))/((Δt/Δx)+α*β*middleterm)
            #qnext=(qj1i*(Δt/Δx)+α*β*qji1*middleterm+Δt*(72.6))/((Δt/Δx)+α*β*middleterm)
        else:
            middleterm=0
            qnext=(qj1i*(Δt/Δx)+α*β*qji1*middleterm+Δt*(estimateie(timecounter)))/((Δt/Δx)+α*β*middleterm)
            #qnext=(qj1i*(Δt/Δx)+α*β*qji1*middleterm+Δt*(72.6))/((Δt/Δx)+α*β*middleterm)
        newspaceflow.append(qnext)
    referencetime+=Δt
    newflow[timecounter]=newspaceflow
columnNames = ['0','Δx','2Δx','3Δx','4Δx','5Δx','6Δx','7Δx','8Δx','9Δx','10Δx','11Δx','12Δx','13Δx','14Δx','15Δx','16Δx','17Δx','18Δx']
df=pd.DataFrame.from_dict(newflow,orient='index', columns = columnNames)
print(df)

          0   Δx  2Δx  3Δx  4Δx  5Δx  6Δx  7Δx  8Δx  9Δx  10Δx  11Δx  12Δx  \
0.000000  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.005556  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.011111  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.016667  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.022222  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.027778  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.033333  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.038889  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.044444  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.050000  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.055556  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   
0.061111  0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  

In [34]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.line(df, x=df.index, y=['17Δx'], title="Flow rates")

observedtime=[]
for i in range(4,len(ObservedUS['Unsteady Plot'])):
    needtoconvert=float(ObservedUS['Unsteady Plot'][i])/60.00
    observedtime.append(needtoconvert)#need to add /3600 was getting error unsupported operand type(s) for /: 'str' and 'int'
#print(observedtime)

observedflow=[]
for i in range(4,len(ObservedUS['Unnamed: 1'])):
    observedprt2y=float(ObservedUS['Unnamed: 1'][i])*.3
    observedflow.append(observedprt2y)

#print(observedflow)
fig.add_trace(go.Scatter(x=observedtime,y=observedflow, mode="lines"))
fig.show()

#fig.write_html(r'C:\Users\AnnaA\Downloads\chapter7fig2.html')
print(observedflow)

[0.0, 0.0, 0.0396, 0.201, 0.2346, 0.1164, 0.0495, 0.0234, 0.012, 0.006599999999999999]


In [ ]:
           
            
14/60